In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入mnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
# direct use real_label,normalize var as 0.5
fake_noise_normal = torch.from_numpy(np.random.normal(loc=0.5, scale=0.5, size=[2048,1,32,32])) # var is 1/4
fake_noise_normal = fake_noise_normal.type(torch.FloatTensor)
fake_noise_normal = fake_noise_normal.to(device)
fake_label = torch.from_numpy(np.random.randint(10,size=[2048]))

fake_noise_uniform = np.sqrt(3)*torch.from_numpy(np.random.uniform(0,1,[2048,1,32,32])) # 1/12
fake_noise_uniform = fake_noise_uniform.type(torch.FloatTensor)
fake_noise_uniform = fake_noise_uniform.to(device)

fake_noise_cauchy = torch.from_numpy(np.random.standard_cauchy(size=[2048,1,32,32])+0.5)
fake_noise_cauchy = fake_noise_cauchy.type(torch.FloatTensor)
fake_noise_cauchy = fake_noise_cauchy.to(device)

fake_noise_laplace = torch.from_numpy(np.random.laplace(loc=0.5, scale=1.0, size=[2048,1,32,32]))/ np.sqrt(8) # var is 2
fake_noise_laplace = fake_noise_laplace.type(torch.FloatTensor)
fake_noise_laplace = fake_noise_laplace.to(device)

fake_noise_gumbel = torch.from_numpy(np.random.gumbel(loc=0.5,scale=1.0,size=[2048,1,32,32]))/ np.sqrt(6.58) #pi^2/6 1.645
fake_noise_gumbel = fake_noise_gumbel.type(torch.FloatTensor)
fake_noise_gumbel = fake_noise_gumbel.to(device)
# from 0.01 - 0.99,0 -1
epsilon_noise = 0.1

In [4]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [5]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10



In [6]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius_noise(radius1,radius2,netD,noise, real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [7]:
# load_model
netD = torch.load('MNIST_NetD-mode11-radius2.pth') 
netV = torch.load('MNIST_NetV-mode11-radius2.pth') 
netG = torch.load('MNIST_NetG-mode11-radius2.pth')

mode = 11
gam3 = 1
net_noise1 = get_noise(DATASET, device, LABEL)

In [21]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET2 = 'FMNIST'
ROOT2    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader2, dataset2 = get_dataloader(
        data_name=DATASET2,
        root=ROOT2,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader2 = iter(dataloader2)
data2, label2 = next(iter_dataloader2)
data_time = time.time() - data_time
real_cpu2 = data2.to(device)
real_label2 = label2.clone().detach()

In [9]:
# data_preprocessing and parameters
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET3 = 'cifar10'
ROOT3    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader3, dataset3 = get_dataloader(
        data_name=DATASET3,
        root=ROOT3,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader3 = iter(dataloader3)
data3, label3 = next(iter_dataloader3)
data_time = time.time() - data_time
real_cpu3 = data3.to(device)
real_label3 = label3.clone().detach()

sample1 = real_cpu3[:,0,:,:]
sample2 = real_cpu3[:,1,:,:]
sample3 = real_cpu3[:,2,:,:]



Files already downloaded and verified


In [13]:
cifar_sample1 = torch.unsqueeze(sample1, dim=1)
cifar_sample2 = torch.unsqueeze(sample2, dim=1)
cifar_sample3 = torch.unsqueeze(sample3, dim=1)

In [22]:
#sample the length of radius,gaussian noise
mode = 11
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.47736796732099385
1.401138159181218
2.9561535164695063
3.0121337990495665
2.7386553095897592


In [23]:
#sample the length of radius,gaussian noise
mode = 1
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

4.537153815209245
5.23957133511916
26.748178209821557
32.330639861830576
99.96444997920295


In [24]:
#sample the length of radius,gaussian noise
net_noise2 = get_noise(DATASET, device, LABEL)
mode = 11
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise2,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise2,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise2,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.47736796732099385
1.401138159181218
2.9561535164695063
3.0121337990495665
2.7386553095897592
4.537153815209245
5.23957133511916
26.748178209821557
32.330639861830576
99.96444997920295


In [25]:
#sample the length of radius,gaussian noise
net_noise3 = get_noise(DATASET, device, LABEL)
mode = 11
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 1

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.1

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.2

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.3

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.4

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.5

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 0.8

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 1.5

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

mode = 1
gam3 = 2

r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise3,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.47736796732099385
1.401138159181218
2.9561535164695063
3.0121337990495665
2.7386553095897592
4.537153815209245
5.23957133511916
26.748178209821557
32.330639861830576
99.96444997920295
0
2.765839142773116
5.0369272117258115
4.778004761978067
4.3213293777179285
0
3.662023408123944
6.299992343401492
6.245351568255141
6.044779732675362
1.537399683419002
4.074175454244463
7.357050196210103
7.473457456920797
7.289163892224513
3.6513293578421617
4.362187321819332
8.358913367858062
8.561278287421548
9.306789251887352
3.944553715035646
4.587486882730347
9.1205562986716
10.511455507989668
13.61040064799729
4.379847372367432
5.0202273189433635
15.975647763699225
20.220357942456488
99.76230643252302
4.732725723104489
5.966830067754133
99.86355705418569
99.76229185276954
99.96444997920295
4.86079017615304
6.339644699386443
99.86355705418569
99.95532160407598
99.96444997920295


In [26]:
# sample 20 times
mode = 11
gam3 = 1

L_mnist = []
L_Fmnist = []
L_cifar1 = []
L_cifar2 = []
L_cifar3 = []

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_mnist.append(r1)
print(max(L_mnist),min(L_mnist),np.mean(L_mnist),np.var(L_mnist))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,real_cpu2,real_label2,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_Fmnist.append(r1)
print(max(L_Fmnist),min(L_Fmnist),np.mean(L_Fmnist),np.var(L_Fmnist))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,cifar_sample1,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_cifar1.append(r1)
print(max(L_cifar1),min(L_cifar1),np.mean(L_cifar1),np.var(L_cifar1))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,cifar_sample2,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_cifar2.append(r1)
print(max(L_cifar2),min(L_cifar2),np.mean(L_cifar2),np.var(L_cifar2))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,cifar_sample3,real_label3,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_cifar3.append(r1)
print(max(L_cifar3),min(L_cifar3),np.mean(L_cifar3),np.var(L_cifar3))


0.540316568044471
0.5443991113699915
0.5069122634716451
0.5201244963798576
0.4950327734082927
0.5135191330280605
0.4981524117556826
0.5271046453478214
0.4849375499977458
0.5195289191193343
0.49718881300646195
0.4931051471828985
0.49911632075461526
0.48085562715708113
0.501638830569329
0.5261407363488888
0.4981522200449367
0.5113648364127901
0.48241504018303205
0.501638830569329
0.5443991113699915 0.48085562715708113 0.5070822137076132 0.0003076671068695391
1.4304572917785916
1.427934471752315
1.3810917660506434
1.7676680543303682
1.5084479517873208
1.3511661097183936
1.5257421075361979
1.631095922696002
1.282575360932699
1.5811340807072847
1.6377048005645563
1.1863380875122325
1.5955380154260435
1.4036606689959317
1.3995781256704112
1.5584088630650463
1.5943468156376848
1.3536886195331073
1.4565139280931556
1.389852430038513
1.7676680543303682 1.1863380875122325 1.4731471735913249 0.01844523024153956
2.3829542695698662
3.1342009652083727
2.9672134611273164
2.741707577305669
2.381987229

In [16]:
#sample the length of radius,gaussian noise
mode = 11
gam3 = 1
L_noise= []
epsilon = 0.05
L_noise1= []
epsilon1 = 0.1

L_noise2= []
epsilon2 = 0.2

L_noise3= []
epsilon3 = 0.3
L_noise4= []
epsilon4 = 0.4
L_noise5= []
epsilon5 = 0.5
L_noise6= []
epsilon6 = 0.6
L_noise7= []
epsilon7 = 0.7
L_noise8= []
epsilon8 = 0.8
L_noise9= []
epsilon9 = 0.9

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise.append(r1)
print(max(L_noise),min(L_noise),np.mean(L_noise),np.var(L_noise))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise1.append(r1)
print(max(L_noise1),min(L_noise1),np.mean(L_noise1),np.var(L_noise1))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise2.append(r1)
print(max(L_noise2),min(L_noise2),np.mean(L_noise2),np.var(L_noise2))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise3.append(r1)
print(max(L_noise3),min(L_noise3),np.mean(L_noise3),np.var(L_noise3))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise4.append(r1)
print(max(L_noise4),min(L_noise4),np.mean(L_noise4),np.var(L_noise4))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise5.append(r1)
print(max(L_noise5),min(L_noise5),np.mean(L_noise5),np.var(L_noise5))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise6.append(r1)
print(max(L_noise6),min(L_noise6),np.mean(L_noise6),np.var(L_noise6))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise7.append(r1)
print(max(L_noise7),min(L_noise7),np.mean(L_noise7),np.var(L_noise7))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise8.append(r1)
print(max(L_noise8),min(L_noise8),np.mean(L_noise8),np.var(L_noise8))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise9.append(r1)
print(max(L_noise9),min(L_noise9),np.mean(L_noise9),np.var(L_noise9))

0.46102832680017475
0.5160427652880878
0.5277001493748398
0.4752129770345196
0.4855332457590862
0.4594694157346817
0.5226475081550293
0.5500404471984581
0.5242136120839525
0.48590126724778354
0.5160427652880878
0.5179696978422783
0.47929652438084214
0.5179700080538412
0.5053523484616554
0.47521347899497746
0.47989202840786044
0.48806000905265395
0.4783314928367095
0.5150791665388671
0.5500404471984581 0.4594694157346817 0.4990498617267193 0.0006006249867063599
0.49911632075461526
0.47736796732099385
0.49562959172940585
0.4748447638350763
0.5088396979481444
0.5185655120808595
0.47580886454475485
0.49659330897944365
0.49058263736818475
0.45286386063398965
0.47076222050955585
0.5201244963798576
0.4890238447799326
0.48709215682457924
0.49778419855623934
0.5012708090806316
0.46355095511570554
0.5119600302518215
0.5006752318201083
0.5176016763535809
0.5201244963798576 0.45286386063398965 0.49250290724337403 0.0003353095990559547
0.46823959219402506
0.4931057676205973
0.4789268783864869
0.506


# other data sets, using same noise to get different r for diff data
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET2 = 'FMNIST'
ROOT2    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader2, dataset2 = get_dataloader(
        data_name=DATASET2,
        root=ROOT2,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader2 = iter(dataloader2)
data2, label2 = next(iter_dataloader2)
data_time = time.time() - data_time
real_cpu2 = data2.to(device)
real_label2 = label2.clone().detach()


device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET3 = 'cifar10'
ROOT3    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader3, dataset3 = get_dataloader(
        data_name=DATASET3,
        root=ROOT3,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader3 = iter(dataloader3)
data3, label3 = next(iter_dataloader3)
data_time = time.time() - data_time
real_cpu3 = data3.to(device)
real_label3 = label3.clone().detach()

sample1 = real_cpu2[:,0,:,:]
sample2 = real_cpu2[:,1,:,:]
sample3 = real_cpu2[:,2,:,:]